## Setup
### Install the packages:

In [1]:
# ! pip install -q -r requirements.txt

### Import libraries :

In [2]:
from datasets import load_dataset
from datasets import Audio
from huggingface_hub import notebook_login
from transformers import AutoFeatureExtractor
from transformers import AutoModelForAudioClassification
from transformers import Trainer
from transformers import TrainingArguments
import evaluate
import numpy as np

/home/victs/Genre_Reveal_Party/GRP/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##  Model Information

In [3]:
model_id = "ntu-spml/distilhubert"

## Data Preparation
### Load Dataset

In [4]:
gtzan = load_dataset("marsyas/gtzan", "all")

In [5]:
# Train Test Split
gtzan = gtzan["train"].train_test_split(seed=42, shuffle=True, test_size=0.1)

In [6]:
gtzan

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})

### Feature Extraction

#### Get Feature Extractor

In [7]:
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)
sampling_rate = feature_extractor.sampling_rate

### Resample Audio Data

In [8]:
gtzan = gtzan.cast_column("audio", Audio(sampling_rate=sampling_rate))

### Extract Featurs

In [9]:
def preprocess_function(examples, max_duration = 30.0):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

In [10]:
gtzan_encoded = gtzan.map(
    preprocess_function,
    remove_columns=["audio", "file"],
    batched=True,
    batch_size=100,
    num_proc=1,
)

### Data Labelling

In [11]:
id2label_fn = gtzan["train"].features["genre"].int2str
id2label = {
    str(i): id2label_fn(i)
    for i in range(len(gtzan_encoded["train"].features["genre"].names))
}
label2id = {v: k for k, v in id2label.items()}

## Instantiate the model

In [12]:
# total number of genres 
num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'classifier.bias', 'classifier.weight', 'projector.weight', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'projector.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Model Training Parameters

In [13]:
model_name = 'GRP'
batch_size = 8
gradient_accumulation_steps = 1
num_train_epochs = 10
training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=False, # Ste to true if using CUDA
    push_to_hub=True,
)

### Training Evaluation Metrics

In [14]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

### Instantiate Trainer

In [15]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

### Train the Model

In [16]:
trainer.train()

: 